In [23]:
%pylab inline
import folium

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['log']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


This notebook shows how to merge planmap's wms-services with external wms. in this example we load a 5-classes geological map of Hokusai quadrangle together with data provided by https://planetarymaps.usgs.gov. Available layers here: https://astrowebmaps.wr.usgs.gov/webmapatlas/Layers/maps.html

In [24]:
from owslib import wfs
PLANMAP = 'https://geoserver.planmap.eu'
PLANMAP_WMS = PLANMAP + '/wms'

In [25]:
import logging as log
log.getLogger().setLevel(log.DEBUG)

In [26]:
id = 'mercury:PM-MER-MS-H05_5cc_01_Geological_Units_5_Classes'

# gather bb information and reduce to the center location
from owslib.wms import WebMapService
wms = WebMapService(PLANMAP_WMS)
bb = wms[id].boundingBox[:4]
loc = np.mean(np.array(bb).reshape(2,2), axis=0)

Here the actual folium map is created

In [29]:
m = folium.Map(zoom_start=3, tiles=None,height=600, width=1200,crs="EPSG4326",location=loc.tolist())

# just as an option for basemap
folium.raster_layers.WmsTileLayer(name="mariner",
                url="https://planetarymaps.usgs.gov/cgi-bin/mapserv?map=/maps/mercury/mercury_simp_cyl.map",
                format="image/png",
                layers='MESSENGER_Mariner', overlay=True, control=True, show=False).add_to(m)


folium.raster_layers.WmsTileLayer(name="messenger_color",
                url="https://planetarymaps.usgs.gov/cgi-bin/mapserv?map=/maps/mercury/mercury_simp_cyl.map",
                format="image/png",
                layers='MESSENGER_Color', overlay=True, control=True, show=False).add_to(m)

folium.raster_layers.WmsTileLayer(
    url=PLANMAP_WMS,
    name='Units 5 classes',
    format='image/png',
    layers=id,
    overlay=True,
    control=True,transparent=True,opacity=0.6
).add_to(m)


folium.raster_layers.WmsTileLayer(
    url=PLANMAP_WMS,
    name='665m DEM',
    format='image/png',
    layers='mercury:USGS_mercury_global_dem_665m',
    overlay=True,
    control=True,transparent=True,opacity=0.6
).add_to(m)


folium.raster_layers.WmsTileLayer(name="labels",
                url="https://wms.wr.usgs.gov/cgi-bin/mapserv?map=/maps/mercury/mercury_nomen_wms.map",
                format='image/png',
                layers='NOMENCLATURE', overlay=True, control=True).add_to(m)

folium.LayerControl().add_to(m)

m